In [3]:
# import statements
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import glob
import os

In [4]:
folder_path = 'csv_letters/*.csv'
x = []
y = []

# Example class names
class_names = ["letterA", "letterB", "letterC","letterD","letterE","letterF","letterG","letterH","letterI","letterK","letterL","letterM","letterN","letterO","letterP","letterQ","letterR","letterS","letterT","letterU","letterV","letterW","letterX","letterY"]

# Create a dictionary mapping class names to numerical labels
class_to_label = {class_name: label for label, class_name in enumerate(class_names)}
# Use glob to get all the csv files in the folder
csv_files = glob.glob(folder_path)

# Initialize an empty list to store the combined DataFrames
combined_dataframes = []
max_len = 0
for i in range(1, len(csv_files), 2):
    # Read the first file into a DataFrame
    df1 = pd.read_csv(csv_files[i], header=0, delimiter=";", usecols=list(range(0,2))+list(range(3,367)), decimal=',')
    if (len(df1)>max_len):
        max_len = len(df1)
print(f"max length is {max_len}")

max length is 315


In [5]:

# Iterate over the files two by two
for i in range(1, len(csv_files), 2):
    # Read the first file into a DataFrame
    df1 = pd.read_csv(csv_files[i], header=0, delimiter=";", usecols=list(range(0,2))+list(range(3,367)), decimal=',')
    # Get the shape of df1
    # print(f"Shape of df1: {df1.shape}")

    #get the class and add it to y
    name = os.path.splitext(csv_files[i])
    classtype = name[0].split('\\')[1][:7] #this takes letterX from the title
    numerical_label = class_to_label.get(classtype, -1)  # -1 if not found
    if numerical_label != -1:
        y.append(numerical_label)
    else:
        print("failed to add to y: " + classtype)

    # If there is a next file, read it and combine with the first
    if i+1 < len(csv_files):
        df2 = pd.read_csv(csv_files[i+1], header=0, delimiter=";", usecols=list(range(0,2))+list(range(3,367)), decimal=',')
        df2 = df2.add_prefix('right_')
    else:
        # If there is no next file, create an empty DataFrame with the same columns as df1
        df2 = pd.DataFrame(columns=df1.columns)

    # Ensure that both DataFrames have the same number of rows by filling NaN values in df2
    if len(df1) > len(df2):
        df2 = df2.reindex(df1.index, fill_value=np.nan)
    elif len(df2) > len(df1):
        df1 = df1.reindex(df2.index, fill_value=np.nan)

    # Concatenate the DataFrames
    combined_df = pd.concat([df1, df2], axis=1)
    combined_df = combined_df.reindex(range(max_len)).fillna(0)
    # Print the shape of combined_df
    print(f"Shape of combined_df: {combined_df.shape}")
    combined_dataframes.append(combined_df)

# Convert the list of DataFrames to a NumPy array
x = np.array(combined_dataframes)
x = np.asarray(x).astype('float32') # meant to fix an error when training the model
y = np.array(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# print(x)
# print(y)



Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of combined_df: (315, 732)
Shape of c

C:\Users\zomaa\AppData\Local\Temp\ipykernel_1760\3567016109.py:33: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_df = combined_df.reindex(range(max_len)).fillna(0)


Shape of combined_df: (315, 732)


In [6]:
model = models.Sequential()
model.add(layers.Conv1D(64, 3, activation='relu', input_shape=(315,732)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(128, 3, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(24, activation='softmax'))

C:\Users\zomaa\IdeaProjects\master\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 313, 64)        │       140,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 156, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 154, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 77, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 75, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,228,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │         1,560 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,453,336 (5.54 MB)

 Trainable params: 1,453,336 (5.54 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(optimizer='adam',  # we used adam during machine learning course
              loss='sparse_categorical_crossentropy',  # Multiclass classification loss
              metrics=['accuracy'])  # Track accuracy during training
print(x.size)
print(y.size)
training_data = model.fit(x_train,y_train,epochs=50)

103069260
447
Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.0405 - loss: 977.2828 
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1036 - loss: 48.1720
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1113 - loss: 7.4829
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.0882 - loss: 3.5975
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.0762 - loss: 3.1584
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.0828 - loss: 3.1044
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.0774 - loss: 3.0926
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0910 - loss: 3.0462
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.1108 - loss: 2.9960
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0897 - loss: 3.0648
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.1075 - loss: 2.9797
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 28m

In [9]:
result = model.evaluate(x_test,y_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0300 - loss: 3.2031     


In [ ]:
import os
import pandas as pd

# Define a function to get the label based on the filename
def get_label(filename):
    letter = filename[6].lower()  # Get the letter from the filename and convert to lowercase
    if letter == 'j':
        return None  # Skip 'j'
    else:
        return ord(letter) - ord('a')  # Convert letter to corresponding integer label

# Specify the folder path where your CSV files are located
folder_path = 'csv_letters_filtered'

# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Iterate through each CSV file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)

        try:
            # Read the first 50 rows from the CSV file
            cols_to_use = list(range(3, 367))
            df = pd.read_csv(file_path, header=0, nrows=50, delimiter=';', usecols=cols_to_use, decimal=',', dtype=float, skiprows=1)

            # Check if the DataFrame has the right number of columns
            if len(df.columns) != 364:  # Modify this number based on your expected number of columns
                print(f"Skipping file '{filename}' as it doesn't have the right number of columns.")
                continue

            # Add a new column with the label
            label = get_label(filename)
            if label is not None:
                df.insert(0, 'Label', label)

            # Add a new column with the filename
            #df['Original_Filename'] = filename

            # Append the data to the combined DataFrame
            combined_df = pd.concat([combined_df, df], ignore_index=True)
        except ValueError:
            print(f"Skipping file '{filename}' as it couldn't be read due to a ValueError.")

# Save the combined data to a new CSV file
combined_df.to_csv('combined_data.csv', index=False)

print("Current working directory:", os.getcwd())

In [ ]:
df = pd.read_csv('combined_data.csv', delimiter=",", decimal=".")
print(df.shape)

In [ ]:
import pandas as pd
import numpy as np

df1 = np.loadtxt('data_glove/data_martijn_zus.csv', delimiter=',', skiprows=1)
df2 = np.loadtxt('data_glove/Data_joren.csv', delimiter=',')

print(df1.shape)
print(df2.shape)

if df1.shape[1] == df2.shape[1]:
    combined_array = np.concatenate((df1, df2), axis=0)
    print("Combined shape:", combined_array.shape)
else:
    print("Number of columns in both arrays must be the same for concatenation.")

In [ ]:
import pandas as pd
import os

# Function to convert letter to corresponding label
def letter_to_label(letter):
    if letter < 'J':
        return ord(letter) - ord('A')
    else:
        return ord(letter) - ord('A') - 1

# Path to the folder containing CSV files
folder_path = 'csv_letters_vake'

# List to store dataframes for each file
dfs = []

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Read CSV file into a dataframe
        df = pd.read_csv(os.path.join(folder_path, filename), delimiter=";", decimal=',', usecols=list(range(3, 367)))
        # Take the first 50 samples
        df = df.head(50)
        print(df.shape)
        # Extract the letter from the filename
        letter = filename.split('_')[0][-4]
        print(letter)
        # Add a label column based on the letter
        df['label'] = letter_to_label(letter)
        df = df[['label'] + [col for col in df.columns if col != 'label']]
        # Append dataframe to the list
        dfs.append(df)

# Concatenate all dataframes into a single dataframe
result_df = pd.concat(dfs)
print(result_df.shape)

# Write the result to a new CSV file
result_df.to_csv('data_glove/data_joren_granfather.csv', index=False)